# MCP Authentication Flow - Illustrated

This notebook illustrates the end-to-end authentication flow for Model Context Protocol (MCP). The flow involves several key steps, including metadata discovery and parsing, client registration, user authentication, token generation, and access validation.

A descripition of this flow is available in the [MCP Authentication Documentation](https://modelcontextprotocol.io/docs/tutorials/security/authorization).

This notebook illustrates the authentication flow with the [GitHub MCP Server].

[GitHub MCP Server]: https://github.com/github/github-mcp-server?tab=readme-ov-file#remote-github-mcp-server

In [9]:
$env:McpServer = "https://api.githubcopilot.com/mcp/"

## Step 1 - Initial Handshake

The initial handshake begins when a client application attempts to access a protected resource on the MCP server. The server responds with 401 Unauthorized status code and provides the client with the MCP metadata URL in the `WWW-Authenticate` header.


In [10]:
curl -s -D - -X POST `
  -H "Content-Type: application/json" `
  -H "Accept: application/json, text/event-stream" `
  -d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "ping"
}' `
 "$env:McpServer"

HTTP/2 401 
content-security-policy: default-src 'none'; sandbox
content-security-policy: default-src 'none'; sandbox
content-type: text/plain; charset=utf-8
content-type: text/plain; charset=utf-8
strict-transport-security: max-age=31536000
strict-transport-security: max-age=31536000
www-authenticate: Bearer error="invalid_request", error_description="No access token was provided in this request", resource_metadata="https://api.githubcopilot.com/.well-known/oauth-protected-resource/mcp/"
www-authenticate: Bearer error="invalid_request", error_description="No access token was provided in this request", resource_metadata="https://api.githubcopilot.com/.well-known/oauth-protected-resource/mcp/"
x-content-type-options: nosniff
x-content-type-options: nosniff
date: Tue, 21 Oct 2025 05:10:50 GMT
date: Tue, 21 Oct 2025 05:10:50 GMT
content-length: 51
content-length: 51
x-github-backend: Kubernetes
x-github-backend: Kubernetes
x-github-request-id: CB07:11ABB6:1923FC5:1EFB149:68F715DA
x-github

## Step 2 - Protected Resource Metadata Discovery

When receiving a 401 response in Step 1, the client extracts the MCP metadata URL from the `WWW-Authenticate` header. Then it sends a GET request to this URL to retrieve the Protected Resource Metadata (PRM) document, which contains information about the authentication server(s) that can be used to authenticate and obtain access tokens.

In [11]:
curl -s -X GET `
  -H "Accept: application/json" `
 "https://api.githubcopilot.com/.well-known/oauth-protected-resource/mcp"

{
  "resource_name": "GitHub MCP Server",
  "resource_name": "GitHub MCP Server",
  "resource": "https://api.githubcopilot.com/mcp",
  "resource": "https://api.githubcopilot.com/mcp",
  "authorization_servers": ["https://github.com/login/oauth"],
  "authorization_servers": ["https://github.com/login/oauth"],
  "bearer_methods_supported": ["header"],
  "bearer_methods_supported": ["header"],
  "scopes_supported": [
  "scopes_supported": [
    "gist",
    "gist",
    "notifications",
    "notifications",
    "public_repo",
    "public_repo",
    "repo",
    "repo",
    "repo:status",
    "repo:status",
    "repo_deployment",
    "repo_deployment",
    "user",
    "user",
    "user:email",
    "user:email",
    "user:follow",
    "user:follow",
    "read:gpg_key",
    "read:gpg_key",
    "read:org",
    "read:org",
    "project"
    "project"
  ]
  ]
}
}


## Step 3 - Authorization Server Discovery

The PRM document should include one or more authorization servers in the `authorization_servers` array. When there is more than one, the client can choose which one to use, but in this case there is only one available. Next, the client retrieves metadata for the authorization server. This could be either OpenID Connect (OIDC) discovery metadata or OAuth 2.0 Authorization Server Metadata, depending on what the authorization server supports. There's no clear indication in the PRM document of which type it is, so the client must try both methods.

According to the [OIDC specification], the client appends `/.well-known/openid-configuration` to the "issuer" URL to get the OIDC discovery document. The client sends a GET request to this URL and receives the OIDC metadata in response. Unfortunately, at this point all we have is the authorization server's URL, so we'll just try to use that as the issuer URL and hope for the best.

[OIDC specification]: https://openid.net/specs/openid-connect-discovery-1_0.html#ProviderConfig

In [12]:
$env:GitHub_MCP_Auth_Server = "https://github.com/login/oauth"
$env:GitHub_MCP_OIDC_Metadata = "$env:GitHub_MCP_Auth_Server/.well-known/openid-configuration"

curl -s -X GET `
  -H "Accept: application/json" `
 "$env:GitHub_MCP_OIDC_Metadata" | jq .

{
  "issuer": "https://github.com",
  "issuer": "https://github.com",
  "jwks_uri": "https://github.com/login/oauth/.well-known/jwks",
  "jwks_uri": "https://github.com/login/oauth/.well-known/jwks",
  "subject_types_supported": [
  "subject_types_supported": [
    "public"
    "public"
  ],
  ],
  "response_types_supported": [
  "response_types_supported": [
    "code",
    "code",
    "id_token"
    "id_token"
  ],
  ],
  "claims_supported": [
  "claims_supported": [
    "sub",
    "sub",
    "aud",
    "aud",
    "exp",
    "exp",
    "nbf",
    "nbf",
    "iat",
    "iat",
    "iss",
    "iss",
    "act"
    "act"
  ],
  ],
  "id_token_signing_alg_values_supported": [
  "id_token_signing_alg_values_supported": [
    "RS256"
    "RS256"
  ],
  ],
  "scopes_supported": [
  "scopes_supported": [
    "openid"
    "openid"
  ]
  ]
}
}


What we are looking for in this metadata is the authorization endpoint, token endpoint, and supported scopes, which will be used in the subsequent steps of the authentication flow. And the scopes are clearly indicated in the `scopes_supported` field of the OIDC metadata.

But we don't see the authorization endpoint or token endpoint in the OIDC metadata. In this case, the best approach, and the one that happens to work with GitHub, is to assume the standard OAuth 2.0 endpoint paths:
- Authorization Endpoint: `{authorization_server_url}/authorize`
- Token Endpoint: `{authorization_server_url}/access_token`

In [13]:
$env:Scopes = "openid"
$env:AuthorizeEndpoint = "$env:GitHub_MCP_Auth_Server/authorize"
$env:TokenEndpoint = "$env:GitHub_MCP_Auth_Server/access_token"

## Step 4 - Client Registration

When the authorization server supports dynamic client registration, the authorization server metadata will include a `registration_endpoint` field. The client can use this endpoint to register itself with the authorization server by sending a POST request with its details, such as redirect URIs and client name. The server responds with a client ID and client secret, which the client will use in subsequent authentication requests.

But GitHub does not support dynamic client registration, so instead we need to register our client application manually in the GitHub Developer Settings to obtain a client ID and client secret.

### Manual Client Registration

Here are the steps to manually register a client application with GitHub:

- Go to your GitHub account settings.
- Navigate to Settings > Developer settings > OAuth Apps.
- Click New OAuth App.
- Fill in the required details (application name, homepage URL, callback URL).
- After registering, GitHub will provide a client_id and a client_secret.

If your application doesn’t have a homepage, you can use a placeholder URL (such as https://example.com) for the “Homepage URL” field when registering your GitHub OAuth App. This field is required, but it does not affect the OAuth flow.

For the authorization callback URL, set the callback URI to a static page (e.g., https://localhost/callback) and, after authorization, GitHub will redirect you there with the code in the URL. You can manually copy the code from the browser’s address bar and use it to exchange for a token.

In [14]:
$env:CallbackUrl = "https://localhost/callback"
$env:CallbackUrlEncoded = "https%3A%2F%2Flocalhost%2Fcallback"


Store the client ID and client secret securely, as they will be needed in the next steps of the authentication flow. For this illustration, the client ID and client secret are stored in a .env file which is not checked into source control.


In [15]:
# Load .env file into the $dotenv hashtable
$dotenv = & "./Load-DotEnv.ps1"

## Step 5 - User Authorization

In this step, the client issues a GET request to the authorization endpoint obtained in Step 3. This request includes parameters such as the client ID, redirect URI, requested scopes, and response type (typically "code" for the authorization code flow). The user is then redirected to the authorization server's login page to authenticate and authorize the client application.

In [16]:
curl -s -D - -X GET `
  -H "Accept: application/json" `
 "$env:AuthorizeEndpoint\?response_type=code&client_id=$($dotenv['ClientId'])&redirect_uri=$env:CallbackUrlEncoded&scope=$env:Scopes"

HTTP/2 302 
date: Tue, 21 Oct 2025 05:12:04 GMT
date: Tue, 21 Oct 2025 05:12:04 GMT
content-type: text/html; charset=utf-8
content-type: text/html; charset=utf-8
content-length: 0
content-length: 0
vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, X-Requested-With,Accept-Encoding, Accept, X-Requested-With
vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, X-Requested-With,Accept-Encoding, Accept, X-Requested-With
location: https://github.com/login?client_id=Ov23liqNxVO8tTBkpYqe&return_to=%2Flogin%2Foauth%2Fauthorize%3Fclient_id%3DOv23liqNxVO8tTBkpYqe%26redirect_uri%3Dhttps%253A%252F%252Flocalhost%252Fcallback%26response_type%3Dcode%26scope%3Dopenid
location: https://github.com/login?client_id=Ov23liqNxVO8tTBkpYqe&return_to=%2Flogin%2Foauth%2Fauthorize%3Fclient_id%3DOv23liqNxVO8tTBkpYqe%26redirect_uri%3Dhttps%253A%252F%252Flocalhost%252Fcallback%26response_type%3Dcode%26scope%3Dopenid
cache-control: no-cache
cache-control: no-cache
set-cookie: _octo=GH1.1.657578288.176102

The response above is a HTTP 302 redirect to your specified callback URL with an authorization code included as a query parameter. The client will receive an authorization code in the query parameter at the redirect URI.

Click on the link in the "Location" header and the redirect will open in a browser. Copy the code from the URL for the next step in the authentication flow.

In [17]:
$env:authCode="024b9deed45be9c0b83c"

Once the client has received the authorization code, it can exchange it for an access token using the token endpoint obtained in Step 3.
The client sends a POST request to the token endpoint with parameters such as the client ID, client secret, authorization code, redirect URI, and grant type (typically "authorization_code"). The server responds with an access token (and optionally a refresh token) that the client can use to access protected resources on behalf of the user.

The response of this request will include an access_token, which you can use to authenticate subsequent requests to the GitHub API.

In [18]:
$body = @{
    client_id = $dotenv['ClientId']
    client_secret = $dotenv['ClientSecret']
    code = $env:authCode
    redirect_uri = $env:CallbackUrl
} | ConvertTo-Json

$response = curl -s -X POST `
  -H "Content-type: application/json" `
  -H "Accept: application/json" `
  -d "$body" `
  "$env:TokenEndpoint"

$env:accessToken = ($response | ConvertFrom-Json).access_token

## Step 6 - Making Authenticated Requests

Here's an example of how to use the access token to make an authenticated request:

In [19]:
curl -s -X POST `
  -H "Authorization: Bearer $env:accessToken" `
  -H "Content-Type: application/json" `
  -H "Accept: application/json, text/event-stream" `
  -d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "clientInfo": {
            "name": "Polyglot Notebook",
            "version": "0.1.0"
        },
        "capabilities": {},
        "protocolVersion": "2025-06-18"
    }
}' `
 "$env:McpServer" | jq .

{
  "jsonrpc": "2.0",
  "jsonrpc": "2.0",
  "id": 1,
  "id": 1,
  "result": {
  "result": {
    "protocolVersion": "2025-06-18",
    "protocolVersion": "2025-06-18",
    "capabilities": {
    "capabilities": {
      "logging": {},
      "logging": {},
      "completion": {},
      "completion": {},
      "prompts": {},
      "prompts": {},
      "resources": {
      "resources": {
        "subscribe": true,
        "subscribe": true,
        "listChanged": true
        "listChanged": true
      },
      },
      "tools": {
      "tools": {
        "listChanged": true
        "listChanged": true
      }
      }
    },
    },
    "serverInfo": {
    "serverInfo": {
      "name": "github-mcp-server",
      "name": "github-mcp-server",
      "version": "github-mcp-server/remote-4435e7b2aa0960d5e55ceef719954f4b5616c253"
      "version": "github-mcp-server/remote-4435e7b2aa0960d5e55ceef719954f4b5616c253"
    },
    },
    "instructions": "The GitHub MCP Server provides tools to interact with